### Silver Layer — Optimization & Maintenance

###### Purpose
Apply physical layout optimizations and maintenance operations to Silver Delta
tables to improve query performance and storage efficiency.

###### Scope
- Optimize Delta file layout
- Apply Z-Ordering for frequent filter columns
- Collect table statistics for query planning



In [0]:
spark.conf.set(
  "fs.azure.account.key.flightdatastorage.blob.core.windows.net",
  ""
)



In [0]:
dbutils.fs.ls(
  "wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market/"
)


[FileInfo(path='wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market/_delta_log/', name='_delta_log/', size=0, modificationTime=1768672662000),
 FileInfo(path='wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market/source_year=2024/', name='source_year=2024/', size=0, modificationTime=1768670910000),
 FileInfo(path='wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market/source_year=2025/', name='source_year=2025/', size=0, modificationTime=1768670950000)]

In [0]:
storage_account_name = "flightdatastorage"
storage_account_key = ""
container_name = "raw"

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    storage_account_key
)

In [0]:
BRONZE_PATH = (
    "wasbs://bronze@flightdatastorage.blob.core.windows.net/"
    "raw_data/data_set/"
)
SILVER_PATH = (
    "wasbs://silver@flightdatastorage.blob.core.windows.net/"
    "flight_market/"
)

###### Delta File Compaction
Compact small Delta files within each partition to improve read performance
and reduce file system overhead.


In [0]:
%sql
OPTIMIZE delta.`wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market/`;



path,metrics
wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market,"List(3, 7, List(172428275, 178779147, 1.76288468E8, 3, 528865404), List(28862083, 134065529, 7.5364804E7, 7, 527553628), 4, null, null, 0, 1, 9, 2, true, 0, 0, 1768672366468, 1768672524045, 4, 3, null, List(0, 0), null, 48, 32, 457338, 0, null)"


###### Z-Order Optimization
Reorder data files to colocate frequently filtered columns and enable
efficient data skipping during query execution.


In [0]:
%sql
OPTIMIZE delta.`wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market/`
ZORDER BY (ORIGIN, DEST, REPORTING_CARRIER);


path,metrics
wasbs://silver@flightdatastorage.blob.core.windows.net/flight_market,"List(1, 2, List(156445271, 156445271, 1.56445271E8, 1, 156445271), List(15484185, 140752106, 7.81181455E7, 2, 156236291), 4, List(minCubeSize(107374182400), List(0, 0), List(5, 685101695), 0, List(2, 156236291), 1, null), null, 0, 1, 5, 3, false, 0, 0, 1768672525451, 1768672662290, 4, 1, null, List(0, 0), null, 48, 32, 95606, 0, null)"
